# Introduction

In order to train on the words in a legislative bill we want to make use of an existing Natural Language Processing (NLP) model in order to come up with a data structure that will encode contextual information.

To this end we use the BERT model, which is one of the modern NLP models that takes into account context.

For this project, we want to take the text from the bill and predict which party sponsored it

## First steps

Load the BERT model using pytorch.

Why pytorch, this is higher level libary for constructing machine learning models.

In [1]:
import fnmatch
import os.path

import pandas as pd
import transformers
from transformers import pipeline
import pandas as pd
import torch
import numpy as np
import json

In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now that we have the model, we want to run inference on the text and get the output token.

In [6]:
def generate_token(summary_json):
    """
    execute the BERT model on the title section of the summary
    :param summary_json: the data from a summary file
    :return: a numpy vector
    """
    text = summary_json['text']
    encoded_input = tokenizer(text, return_tensors='pt')
    print("encoded input")
    print(encoded_input)
    print("model output")
    output = model(**encoded_input)
#    print(type(output))
    last_shape = output.last_hidden_state.shape
    elements = np.cumproduct(last_shape)
    last_layer_vector = torch.from_numpy(output.last_hidden_state.detach().numpy().reshape(max(elements)))
#    print(type(last_layer_vector))
    return last_layer_vector

In [7]:
parent_path = os.path.dirname(os.getcwd())
search_path = os.path.join(parent_path, "data", "extracted")
token_path = os.path.join(parent_path, "data", "tokenized")

In [ ]:
import pickle
import fnmatch
import traceback
with open("processed_files.txt",'w') as p_list, open("summary_pkl_files.txt", 'w') as s_files:
    for root, dirs, files in os.walk(search_path):
        for f in files:
            if fnmatch.fnmatch(f, "summary_bill_text*.json"):
                with open(os.path.join(root, f), 'r') as s_file:
                    print(f'reading {f}')
                    try:
                        summary = json.load(s_file)
                        encoding = generate_token(summary)
                        print(f'{len(encoding)} of type {type(encoding)}')
                        summary["input_full"] = encoding
                        df = pd.DataFrame(summary)
                        output_path = os.path.join(token_path, f.replace(".json",".pkl"))
                        print(f'saving {output_path}')
                        df.to_pickle(output_path,compression="gzip",protocol=pickle.DEFAULT_PROTOCOL)
                        p_list.write(f'{f}\n')
                        s_files.write(f'{output_path}]\n')
                    except json.JSONDecodeError:
                        with open("json_error.txt", 'a') as err_file:
                            err_file.write(s_file.name+"\n")
                    except Exception as e:
                        traceback.print_exc()
                        with open("general_error.txt", 'a') as err_file:
                            err_file.write(f'{s_file.name} : {str(e)}\n')




These tensors all have different sizes, so running a cosine similarity will not work.
I will try to compress these down to a standard size.
Let's try this  : https://newbedev.com/downsample-a-1d-numpy-array

Let's reload and the dfs and save a resampled version of the encoding

In [8]:
import numpy as np
from scipy.signal import resample
from scipy.interpolate import interp1d

def ResampleLinear1D(original, targetLen):
    original = np.array(original, dtype=np.float)

    index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.float)
    index_floor = np.array(index_arr, dtype=np.int) #Round down

    index_ceil = index_floor + 1
    index_rem = index_arr - index_floor #Remain

    val1 = original[index_floor]
    val2 = original[index_ceil % len(original)]
    interp = val1 * (1.0-index_rem) + val2 * index_rem
    assert(len(interp) == targetLen)
    return interp

In [12]:
import pickle
print(f'tokenized files are in {token_path}')
import fnmatch
for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary*bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            print(f"processsing {f}")
            df = pd.read_pickle(pkl_file_path, compression="gzip")
    #        print(df)
            shrunk = ResampleLinear1D(df["input_full"], 2048)
    #        print(shrunk)
            # "status": 2, "party": 1
            df2 = pd.DataFrame({"status": df["status"][0],
                                "party": df["party"][0],
                                "input_shrunk": [shrunk]})
    #        print(df2.shape)
            df2.to_pickle(pkl_file_path.replace(".pkl","-shrunk.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

tokenized files are in C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\data\tokenized
processsing summary_bill_text_1811_1392984.pkl
processsing summary_bill_text_1811_1392985.pkl
processsing summary_bill_text_1811_1392986.pkl
processsing summary_bill_text_1811_1392988.pkl
processsing summary_bill_text_1811_1392991.pkl
processsing summary_bill_text_1811_1392992.pkl
processsing summary_bill_text_1811_1392995.pkl
processsing summary_bill_text_1811_1392996.pkl
processsing summary_bill_text_1811_1393001.pkl


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1811_1393003.pkl
processsing summary_bill_text_1811_1393010.pkl
processsing summary_bill_text_1811_1393013.pkl
processsing summary_bill_text_1811_1393014.pkl
processsing summary_bill_text_1811_1393017.pkl
processsing summary_bill_text_1811_1393019.pkl
processsing summary_bill_text_1811_1393020.pkl
processsing summary_bill_text_1811_1393023.pkl
processsing summary_bill_text_1811_1393024.pkl


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1811_1393030.pkl
processsing summary_bill_text_1811_1393032.pkl
processsing summary_bill_text_1811_1393033.pkl
processsing summary_bill_text_1811_1393034.pkl
processsing summary_bill_text_1811_1393038.pkl
processsing summary_bill_text_1811_1393040.pkl
processsing summary_bill_text_1811_1393041.pkl
processsing summary_bill_text_1811_1393044.pkl
processsing summary_bill_text_1811_1393047.pkl


...

C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1959_1546211.pkl
processsing summary_bill_text_1959_1546220.pkl
processsing summary_bill_text_1959_1546222.pkl
processsing summary_bill_text_1959_1546250.pkl
processsing summary_bill_text_1959_1546275.pkl
processsing summary_bill_text_1959_1546278.pkl
processsing summary_bill_text_1959_1546285.pkl
processsing summary_bill_text_1959_1546313.pkl


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1959_1546316.pkl
processsing summary_bill_text_1959_1546379.pkl
processsing summary_bill_text_1959_1546390.pkl
processsing summary_bill_text_1959_1546395.pkl
processsing summary_bill_text_1959_1546412.pkl
processsing summary_bill_text_1959_1546417.pkl
processsing summary_bill_text_1959_1546431.pkl
processsing summary_bill_text_1959_1546443.pkl
processsing summary_bill_text_1959_1546461.pkl
processsing summary_bill_text_1959_1575848.pkl
processsing summary_bill_text_1959_1579816.pkl


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

Doing the interpolation, while convienent, may be supressing important information. Let's try using max pooling instead.
This to get a consisent on the output, we need to pad the token vector so that we have a multiple of 2048 elements.

In [14]:
import math
def max_pool_to_size(encoding: torch.Tensor, target_size):
    """
    Take the raw bert encoding and use max pool 1d to get to the target size,
    will need to pad the encoding.
    MaxPool1D takes in two variables that control how the max-pooling works, stride and kernel size. For simpliclity we will
    take stride to equal kernel size.
    :param encoding: the raw bert encoding
    :param target_size: the number of elements in the output vector
    :return:
    """

    enc_length = len(encoding)
    print(enc_length)
    if enc_length % target_size == 0:
        kernel_size = int(enc_length / target_size)
        padding = 0

    else:
        kernel_size = math.floor(enc_length / target_size) + 1
        padding = (kernel_size * target_size) - enc_length

    if padding == 0:
        mp_tensor = encoding
        padded_size = enc_length
    elif padding < 0:
        print(f"oops. got padding {padding} for kernel_size {kernel_size}, enc_length {enc_length}")
        raise ValueError("oops")
    else:
        # concatenate on an tensor of zeros to the end to
        mp_tensor = torch.cat((encoding, torch.Tensor(np.zeros(padding))))
        padded_size = len(mp_tensor)
    # print(f'original length {enc_length}, padding {padding}, padded size {padded_size}, kernel = {kernel_size}')
    maxpooler = torch.nn.MaxPool1d(kernel_size, stride=kernel_size, padding=0)
    return maxpooler(mp_tensor.resize(1,padded_size))


In [15]:
for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary_bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            print(f"processsing {f}")
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            shrunk = max_pool_to_size(torch.Tensor(df["input_full"]), 2048)
            # "status": 2, "party": 1
            df2 = pd.DataFrame({"status": df["status"][0],
                                "party": df["party"][0],
                                "input_maxpool": [shrunk.reshape(2048)]})
    #        print(df2.shape)
            df2.to_pickle(pkl_file_path.replace(".pkl","-maxpool.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

processsing summary_bill_text_1811_1392984.pkl
102144
processsing summary_bill_text_1811_1392985.pkl
110592
processsing summary_bill_text_1811_1392986.pkl
116736
processsing summary_bill_text_1811_1392988.pkl
203520
processsing summary_bill_text_1811_1392991.pkl


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


135936
processsing summary_bill_text_1811_1392992.pkl
181248
processsing summary_bill_text_1811_1392995.pkl
25344
processsing summary_bill_text_1811_1392996.pkl
267264
processsing summary_bill_text_1811_1393001.pkl


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


52224
processsing summary_bill_text_1811_1393003.pkl
72960
processsing summary_bill_text_1811_1393010.pkl
177408
processsing summary_bill_text_1811_1393013.pkl
89088
processsing summary_bill_text_1811_1393014.pkl
96000
processsing summary_bill_text_1811_1393017.pkl
71424


...

c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


42240
processsing summary_bill_text_1959_1546443.pkl
76800
processsing summary_bill_text_1959_1546461.pkl
291072
processsing summary_bill_text_1959_1575848.pkl


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


291072
processsing summary_bill_text_1959_1579816.pkl
178176


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
%%capture
import pickle
for root,dirs, files in os.walk(token_path):
    for f in files:
        target_size = 512
        if not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*") and not fnmatch.fnmatch(f, '*.tar.gz'):
            pkl_file_path = os.path.join(root, f)
            print(f"processsing {f}")
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            shrunk = max_pool_to_size(torch.Tensor(df["input_full"]), target_size)
            # "status": 2, "party": 1
            df2 = pd.DataFrame({"status": df["status"][0:target_size],
                                "party": df["party"][0:target_size],
                                "input_maxpool": shrunk.reshape(target_size)})
    #        print(df2.shape)
            df2.to_pickle(pkl_file_path.replace(".pkl","-maxpool-512.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

Using 2048 may be loosing too much information. so let's try to double that to 4096

In [18]:
import pickle
print(f'tokenized files are in {token_path}')
import fnmatch
target_size = 4096
for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary*bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            full_encoding = df["input_full"]
            print(f"processsing {f} : {len(full_encoding)}")

            if len(full_encoding) >= target_size:
        #        print(df)
                shrunk = ResampleLinear1D(full_encoding, target_size)
        #        print(shrunk)
                # "status": 2, "party": 1
                df2 = pd.DataFrame({"status": df["status"][0],
                                    "party": df["party"][0],
                                    "input_shrunk": [shrunk]})
        #        print(df2.shape)
                df2.to_pickle(pkl_file_path.replace(".pkl",f"-shrunk-{target_size}.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

tokenized files are in C:\Users\benja\git-projects\bitbucket\nlp_legislation_prediction\data\tokenized
processsing summary_bill_text_1811_1392984.pkl : 102144
processsing summary_bill_text_1811_1392985.pkl : 110592
processsing summary_bill_text_1811_1392986.pkl : 116736
processsing summary_bill_text_1811_1392988.pkl : 203520
processsing summary_bill_text_1811_1392991.pkl : 135936


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1811_1392992.pkl : 181248
processsing summary_bill_text_1811_1392995.pkl : 25344
processsing summary_bill_text_1811_1392996.pkl : 267264
processsing summary_bill_text_1811_1393001.pkl : 52224
processsing summary_bill_text_1811_1393003.pkl : 72960
processsing summary_bill_text_1811_1393010.pkl : 177408
processsing summary_bill_text_1811_1393013.pkl : 89088
processsing summary_bill_text_1811_1393014.pkl : 96000
processsing summary_bill_text_1811_1393017.pkl : 71424


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1811_1393019.pkl : 303360
processsing summary_bill_text_1811_1393020.pkl : 180480
processsing summary_bill_text_1811_1393023.pkl : 139008
processsing summary_bill_text_1811_1393024.pkl : 236544
processsing summary_bill_text_1811_1393030.pkl : 300288
processsing summary_bill_text_1811_1393032.pkl : 56064


...

C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1959_1546390.pkl : 159744
processsing summary_bill_text_1959_1546395.pkl : 197376
processsing summary_bill_text_1959_1546412.pkl : 251904
processsing summary_bill_text_1959_1546417.pkl : 104448
processsing summary_bill_text_1959_1546431.pkl : 42240
processsing summary_bill_text_1959_1546443.pkl : 76800
processsing summary_bill_text_1959_1546461.pkl : 291072


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

processsing summary_bill_text_1959_1575848.pkl : 291072
processsing summary_bill_text_1959_1579816.pkl : 178176


C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  original = np.array(original, dtype=np.float)
C:\Users\benja\AppData\Local\Temp\ipykernel_27944\376862322.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.fl

And the same for maxpool1d\

In [19]:
for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary_bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            full_encoding = df["input_full"]
            print(f"processsing {f} : {len(full_encoding)}")

            if len(full_encoding) >= target_size:
                shrunk = max_pool_to_size(torch.Tensor(full_encoding), target_size)
                # "status": 2, "party": 1
                df2 = pd.DataFrame({"status": df["status"][0],
                                    "party": df["party"][0],
                                    "input_maxpool": [shrunk.reshape(target_size)]})
        #        print(df2.shape)
                df2.to_pickle(pkl_file_path.replace(".pkl",f"-maxpool-{target_size}.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

processsing summary_bill_text_1811_1392984.pkl : 102144
102144
processsing summary_bill_text_1811_1392985.pkl : 110592
110592
processsing summary_bill_text_1811_1392986.pkl : 116736
116736
processsing summary_bill_text_1811_1392988.pkl : 203520
203520


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


processsing summary_bill_text_1811_1392991.pkl : 135936
135936
processsing summary_bill_text_1811_1392992.pkl : 181248
181248
processsing summary_bill_text_1811_1392995.pkl : 25344
25344
processsing summary_bill_text_1811_1392996.pkl : 267264


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


267264
processsing summary_bill_text_1811_1393001.pkl : 52224
52224
processsing summary_bill_text_1811_1393003.pkl : 72960
72960
processsing summary_bill_text_1811_1393010.pkl : 177408
177408
processsing summary_bill_text_1811_1393013.pkl : 89088
89088
processsing summary_bill_text_1811_1393014.pkl : 96000


...

c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


251904
processsing summary_bill_text_1959_1546417.pkl : 104448
104448
processsing summary_bill_text_1959_1546431.pkl : 42240
42240
processsing summary_bill_text_1959_1546443.pkl : 76800
76800


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


processsing summary_bill_text_1959_1546461.pkl : 291072
291072
processsing summary_bill_text_1959_1575848.pkl : 291072
291072
processsing summary_bill_text_1959_1579816.pkl : 178176
178176


c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\benja\git-projects\bitbucket\nlp_legislation_prediction\venvs3\lib\site-packages\torch\_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [20]:
%%capture
# suppress output
target_size = 2048*4
for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary*bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            full_encoding = df["input_full"]
            print(f"processsing {f} : {len(full_encoding)}")

            if len(full_encoding) >= target_size:
        #        print(df)
                shrunk = ResampleLinear1D(full_encoding, target_size)
        #        print(shrunk)
                # "status": 2, "party": 1
                df2 = pd.DataFrame({"status": df["status"][0],
                                    "party": df["party"][0],
                                    "input_shrunk": [shrunk]})
        #        print(df2.shape)
                df2.to_pickle(pkl_file_path.replace(".pkl",f"-shrunk-{target_size}.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

In [21]:
%%capture
for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary_bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            full_encoding = df["input_full"]
            print(f"processsing {f} : {len(full_encoding)}")

            if len(full_encoding) >= target_size:
                shrunk = max_pool_to_size(torch.Tensor(full_encoding), target_size)
                # "status": 2, "party": 1
                df2 = pd.DataFrame({"status": df["status"][0],
                                    "party": df["party"][0],
                                    "input_maxpool": [shrunk.reshape(target_size)]})
        #        print(df2.shape)
                df2.to_pickle(pkl_file_path.replace(".pkl",f"-maxpool-{target_size}.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

In [23]:
%%capture
target_size=2048*8
for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary*bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            full_encoding = df["input_full"]
            print(f"processsing {f} : {len(full_encoding)}")

            if len(full_encoding) >= target_size:
        #        print(df)
                shrunk = ResampleLinear1D(full_encoding, target_size)
        #        print(shrunk)
                # "status": 2, "party": 1
                df2 = pd.DataFrame({"status": df["status"][0],
                                    "party": df["party"][0],
                                    "input_shrunk": [shrunk]})
        #        print(df2.shape)
                df2.to_pickle(pkl_file_path.replace(".pkl",f"-shrunk-{target_size}.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)

for root,dirs, files in os.walk(token_path):
    for f in files:
        if fnmatch.fnmatch(f, "summary_bill_text*") and not fnmatch.fnmatch(f, "*shrunk*") and not fnmatch.fnmatch(f, "*maxpool*"):
            pkl_file_path = os.path.join(root, f)
            df = pd.read_pickle(pkl_file_path, compression="gzip")
            full_encoding = df["input_full"]
            print(f"processsing {f} : {len(full_encoding)}")

            if len(full_encoding) >= target_size:
                shrunk = max_pool_to_size(torch.Tensor(full_encoding), target_size)
                # "status": 2, "party": 1
                df2 = pd.DataFrame({"status": df["status"][0],
                                    "party": df["party"][0],
                                    "input_maxpool": [shrunk.reshape(target_size)]})
        #        print(df2.shape)
                df2.to_pickle(pkl_file_path.replace(".pkl",f"-maxpool-{target_size}.pkl"),compression="gzip", protocol=pickle.DEFAULT_PROTOCOL)